In [7]:
# pip install utils


In [4]:
from xgboost import XGBRegressor
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import sklearn

import tensorflow as tf

from utils import *
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [9]:
pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00


In [10]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="NeuralNetwork", entity="team-red")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Problem at: <ipython-input-10-393deb9a2fd2> 4 <cell line: 4>


CommError: ignored

In [ ]:
# Importing the dataset
dataset = pd.read_csv('data.csv')

# Shuffle the dataset
dataset = dataset.sample(frac=1)

# Drop ZIP code column
dataset = dataset.drop(['ZIP'], axis=1)

# Convert #3 START_TIME (pandas Series Object) to datetime without date
dataset['START_TIME'] = pd.to_datetime(dataset['START_TIME']).apply(lambda x: x.value)

# drops index
try:
    dataset = dataset.drop(['INDEX'], axis=1)
except:
    pass

dataset.head()
dataset.info()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
x = dataset.iloc[:,0:-1].values
y = dataset.iloc[:, -1].values
x,y

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
x

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, random_state=1)

In [ ]:
wandb.config = {
  "learning_rate": 1,
  "epochs": 350,
  "batch_size": 12
}

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256/2, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256/4, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=[r2metrics, soft_acc, 'mean_absolute_error', 'mean_squared_error'])
NN_model.summary()

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(x_train, y_train, epochs=350, batch_size=12, validation_split = .05, callbacks=[checkpoint, WandbCallback()])

22/22 [==============================] - ETA: 0s - loss: 3.9347 - r2metrics: 0.6138 - soft_acc: 0.1061 - mean_absolute_error: 3.9347 - mean_squared_error: 25.9350
Epoch 101: val_loss did not improve from 4.07710
22/22 [==============================] - 0s 6ms/step - loss: 3.9347 - r2metrics: 0.6138 - soft_acc: 0.1061 - mean_absolute_error: 3.9347 - mean_squared_error: 25.9350 - val_loss: 4.7208 - val_r2metrics: 0.5915 - val_soft_acc: 0.0417 - val_mean_absolute_error: 4.7208 - val_mean_squared_error: 35.5486 - _timestamp: 1654745425.0000 - _runtime: 22.0000
Epoch 102/350
13/22 [================>.............] - ETA: 0s - loss: 3.0101 - r2metrics: 0.7185 - soft_acc: 0.1282 - mean_absolute_error: 3.0101 - mean_squared_error: 16.8265
Epoch 102: val_loss improved from 4.07710 to 3.99400, saving model to Weights-102--3.99400.hdf5
22/22 [==============================] - 1s 27ms/step - loss: 3.1820 - r2metrics: 0.7114 - soft_acc: 0.1250 - mean_absolute_error: 3.1820 - mean_squared_error: 18.3

In [ ]:
score = NN_model.evaluate(x_test, y_test)
print(score)